## ENSE 496 AD and AC project - Recommendation System with using User based Collaborative Filtering Algorithm

In this project:

In [70]:
#Import 
import numpy as np
import math
import pandas as pd


In [87]:
# Function that calculate simlary by Cosine Similarity between two users
def simlaryCosDist(user1, user2):
    xSum = 0
    ySum = 0
    xySum = 0
    for rate1 in user1:
        for rate2 in user2:
            if rate1[0] == rate2[0]:
                xySum += rate1[1] * rate2[1]
                ySum += rate2[1] ** 2
                xSum += rate1[1] ** 2
    if xySum == 0: return 0
    return xySum / ((ySum * xSum) ** 0.5)


In [45]:
# generate user dictory
def genUserDic(datas):
    userRating = {}
    itemUser = {}
    for row in datas:
        rating = (row[1],row[2])
        if row[0] in userRating: userRating[row[0]].append(rating)
        else: userRating[row[0]] = [rating]

        if row[1] in itemUser: itemUser[row[1]].append(row[0])
        else: itemUser[row[1]] = [row[0]]

    return userRating, itemUser

In [46]:
# Get the near neighbors 
def getNearNeighbors(userId, userDic, itemDic):
    neighborsList = []
    #print(userDic)
    #print(userId)
    #print(userDic[userId])
    
    for ratings in userDic[userId]:
        for neighbor in itemDic[ratings[0]]:
            if neighbor != userId and neighbor not in neighborsList:
                neighborsList.append(neighbor)
    
    neighborsSimilary = []
    for neighbor in neighborsList:
        similary = simlaryCosDist(userDic[userId], userDic[neighbor])
        neighborsSimilary.append([similary, neighbor])
    neighborsSimilary.sort(reverse = True)
    return neighborsSimilary
    


In [47]:
def recommendation(userId,userDic, itemDic, k = 5):
    #print(itemDic[userId])
    neighbors = getNearNeighbors(userId, userDic, itemDic)[:k]
    #print("Neighbors",neighbors)
    recommendDic = {}
    for neighbor in neighbors:     #neighbor: [similary, neighbor Id]
        neighborUserId = neighbor[1]
        itemRating = userDic[neighborUserId] #[product Id , rating]
        for rating in itemRating:
            if rating[0] in recommendDic: recommendDic[rating[0]] += neighbor[0]
            else: recommendDic[rating[0]] = neighbor[0]

    recommendList = []
    for ele in recommendDic:  #ele: {product Id: sum of similary}
        recommendList.append([recommendDic[ele], ele])

    recommendList.sort(reverse=True)

    return [i[1] for i in recommendList], neighbors


In [85]:
# main function to test
# Load the data set 
datas = pd.read_csv("ratings_Beauty.csv")
test = datas[:1000000]
test = np.array(test)
#print(test[0][0])
userDic, itemDic = genUserDic(test)
#print(userDic)
#print(itemDic)
#print(userDic[test[0][0]])


In [88]:
testRecomm = test[:5]
for t in testRecomm:
    recommendList, neighbors = recommendation(t[0], userDic, itemDic)
    print("User ID", t[0] , "--> recommend item ID List: ", recommendList)


User ID A39HTATAQ9V7YF --> recommend item ID List:  ['B002OVV7F0', 'B002XQ1YL8', 'B002L6NZHG', 'B001SYYMF6', 'B001PTFWSA', 'B001CTUX5K', 'B000OWHKMU', 'B000LZLILE', 'B000F1OVRC', 'B0009OAI18']
User ID A3JM6GV9MNOF9X --> recommend item ID List:  ['0558925278']
User ID A1Z513UWSAAO0F --> recommend item ID List:  ['0558925278']
User ID A1WMRR494NWEWV --> recommend item ID List:  []
User ID A3IAAVS479H7M7 --> recommend item ID List:  ['B0002DNZAC', 'B00139FCVO', 'B002NN6Z38', 'B0018HPFNG', 'B00126IXLO', 'B000UYIKM6', 'B000L596FE', 'B000HRVC5I', 'B0006FMR32']


In [74]:
def getPrecision(userId, recommendList,userDic):
    user = [i[0] for i in userDic[userId]]
    recommand = [i for i in recommendList]
    count = 0.0
    if len(user) >= len(recommand):
        for item in recommand:
            if item in user: count += 1
        precision = count / len(recommand)
    else:
        for item in user:
            if item in recommand: count += 1
        precision = count / len(user)
    return precision



In [76]:
p = getPrecision(userId, recommendList,userDic)
p

0.6666666666666666